In [12]:
import numpy as np 
import pandas as pd
import re
import gc
import os
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

In [13]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [14]:
def detect(tokens):
    return [t for t in tokens if t in valid_forms]
    
def replace_blank(tokens):
    return [blank if t in valid_forms else t for t in tokens]

def create_windows(tokens, window_size=3):
    X = []
    for i, word in enumerate(tokens):
        if word == blank:
            window = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
            window = ' '.join(window)
            X.append(window)    
    return X

In [15]:
df = pd.read_csv('Tamil_first_ready_for_sentiment.csv',sep='\t',names=['category','text'])
print(df.shape)

df['text'] = df.text.str.lower().str.replace(r'['+string.digits+string.punctuation+']', ' ')
df['text'] = df['text'].apply(lambda x: x.split())

(15744, 2)


In [16]:
y=df['category']
#y = pd.get_dummies(df['category']).values

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), y, random_state=5, test_size=0.2)
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
y_train=pd.get_dummies(y_train).values
y_test=pd.get_dummies(y_test).values
diz_label = {}
for i,label in enumerate(df.category.factorize()[1]):
    diz_label[i] = label
    
diz_label

{0: 'Negative ',
 1: 'Positive ',
 2: 'Mixed feelings ',
 3: 'not-Tamil ',
 4: 'unknown state '}

In [17]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=50)
sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, maxlen=50)

In [18]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(5, activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train,
                    batch_size=1024,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,class_weight=class_weights)

W0131 16:28:12.253907 140269875492608 deprecation.py:323] From /home/bharaj/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 11335 samples, validate on 1260 samples
Epoch 1/10
11335/11335 [==============================] - 7s 595us/step - loss: 1.3379 - acc: 0.6211 - val_loss: 1.1087 - val_acc: 0.6794
Epoch 2/10
11335/11335 [==============================] - 1s 69us/step - loss: 1.0603 - acc: 0.6730 - val_loss: 1.0260 - val_acc: 0.6794
Epoch 3/10
11335/11335 [==============================] - 1s 69us/step - loss: 1.0234 - acc: 0.6730 - val_loss: 1.0076 - val_acc: 0.6794
Epoch 4/10
11335/11335 [==============================] - 1s 70us/step - loss: 0.9946 - acc: 0.6730 - val_loss: 0.9862 - val_acc: 0.6794
Epoch 5/10
11335/11335 [==============================] - 1s 67us/step - loss: 0.9387 - acc: 0.6730 - val_loss: 0.9513 - val_acc: 0.6794
Epoch 6/10
11335/11335 [==============================] - 1s 72us/step - loss: 0.8477 - acc: 0.6783 - val_loss: 0.9426 - val_acc: 0.6802
Epoch 7/10
11335/11335 [==============================] - 1s 71us/step - loss: 0.7597 - acc: 0.7274 - val_loss: 0.9807 - val_acc

In [21]:
score = model.evaluate(X_test, y_test,
                       batch_size=256, verbose=1)
print('Test accuracy:', score[1])

3149/3149 [==============================] - 0s 86us/step
Test accuracy: 0.6348046994126005


In [22]:
preds = model.predict(X_test)

In [28]:
# {0: 'Negative ',
#  1: 'Positive ',
#  2: 'Mixed feelings ',
#  3: 'not-Tamil ',
#  4: 'unknown state '}

In [23]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(preds,axis=1)))

              precision    recall  f1-score   support

           0       0.14      0.10      0.12       377
           1       0.30      0.11      0.16       424
           2       0.71      0.91      0.80      2075
           3       0.67      0.28      0.39       100
           4       0.00      0.00      0.00       173

   micro avg       0.63      0.63      0.63      3149
   macro avg       0.36      0.28      0.29      3149
weighted avg       0.54      0.63      0.57      3149



/home/bharaj/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=25)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize = 14)

    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)

In [ ]:
# cnf_matrix = confusion_matrix([diz_label[i] for i in np.argmax(y_test, axis=1)], 
#                               [diz_label[i] for i in np.argmax(preds, axis=1)])

# plt.figure(figsize=(7,7))
# plot_confusion_matrix(cnf_matrix, classes=list(diz_label.values()), title="Confusion matrix DME")
# plt.show()